# Assignment 6
You have to replicate all the maps in this JN using folium.

In [ ]:
First we install folium

!pip install folium

In [3]:
import folium as fm

In [4]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))


Then also install geopandas

!pip install geopandas
!pip install --upgrade numpy
!pip install --upgrade numpy scipy
!pip install --upgrade geopandas
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import geopandas as gpd

In [13]:
# Get data of labor
sex_work = pd.read_stata('C:\\Users\\HP\\Documents\\GitHub\\QLAB_Summer_Python\\MediaLab_Summer_Python\\_data\\8_trab_sex_20.dta')
sex_work[ 'dpt_code' ] = sex_work.cod_ubigeo.str[ :2 ].copy()
sex_work[ 'prov_code' ] = sex_work.cod_ubigeo.str[ :4 ].copy()

# Sex work
women_work = sex_work[ sex_work.sex == 'Mujer' ].copy().reset_index( drop = True )

In [4]:
# get data from lima
women_work[ 'month' ] = pd.to_datetime( women_work.month , format = '%B' ) \
                                    .dt.strftime( '%m' ) \
                                    .astype( int )

In [40]:
# Sort by department and month
women_work.sort_values([ 'dpt_code', 'month'], inplace = True )

# Get the total number of women workers by dpt
dpt_women_work = women_work.groupby( [ 'dpt_code', 'month'], as_index = False )[['empl']] \
                            .sum() \
                            .rename( columns = {'empl' :'women_empl'})

# Sort by dpt code and month
dpt_women_work.sort_values([ 'dpt_code', 'month'], inplace = True )

In [41]:
df2 = dpt_women_work.groupby( ['dpt_code'],as_index = False )[['women_empl']].mean()

## Shapefile

# Map 1

In [25]:
dpt_shp = gpd.read_file(r"C:\Users\HP\Documents\GitHub\QLAB_Summer_Python\MediaLab_Summer_Python\_data\INEI_LIMITE_DEPARTAMENTAL\INEI_LIMITE_DEPARTAMENTAL.shp")

In [28]:
print(dpt_shp.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [42]:
df3 = dpt_shp.merge(df2, left_on='CCDD', right_on='dpt_code')

df3.plot( column='women_empl', cmap='Reds', figsize=(20, 20), linestyle='--',
                      edgecolor='black', 
                      legend = True) 

# Map 2

In [59]:
import folium

!pip install folium

In [ ]:
folium.Choropleth(
    geo_data=df3,
    name='choropleth',
    data=df3,
    columns=['dpt_code', 'women_empl'],
    key_on='feature.properties.dpt_code',
    fill_color='YlOrRd',
    fill_opacity=0.9,
    line_opacity=0.2,
    legend_name='Women Employment (%)'
).add_to(folium_map)

folium_map

In [ ]:
import folium
from folium.plugins import MarkerCluster

import pandas as pd
import geopandas as gpd

# Calculate the map center
map_center_two = [df3.geometry.centroid.y.mean(), df3.geometry.centroid.x.mean()]

# Create a Folium map centered at the calculated location
folium_map_two = folium.Map(location=map_center_2, zoom_start=5)

# Check unique values in 'women_empl' column
unique_values = df3['women_empl'].unique()
print("Unique values in 'women_empl':", unique_values)

# Manually specify bin edges based on unique values
bins = [0] + sorted(unique_values)


In [70]:
df4 = sex_work.groupby( ['dpt_code', 'month', 'sex'], as_index = False )[['empl']].sum() \
        .pivot( index = [ 'dpt_code', 'month' ] , columns = 'sex',values='empl') \
        .reset_index()

In [72]:
df4[ 'prop_wom' ] = ( df4.Mujer * 100 / df4.Hombre )

In [73]:
df5 = dpt_shp.merge( df4, left_on = 'CCDD', right_on = 'dpt_code'  )

In [ ]:
fig, axis = plt.subplots( nrows = 4, ncols= 3, figsize = ( 15, 15 ) )

idx = 0
for i in range( 4 ):
    for j in range ( 3 ):
        
        
        ax = axis[ i ][ j ]
        
        month = df5.month.unique()[ idx ]
        
        df6 = df5[ df5.month == month ]
        
        df6.plot( column='prop_wom', 
                  cmap='Reds', 
                  linestyle='--',
                  edgecolor='black', 
                  legend = True, 
                  ax = ax 
                )
        
        ax.set_title( month )
        
        idx = idx + 1

In [74]:
from textwrap import wrap

In [75]:
# Inverting colour map
cmap = plt.cm.OrRd

In [77]:
df6.loc[ (df6.NOMBDEP == 'LIMA'), 'prop_wom' ] = np.nan

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
df6.plot( ax = ax, 
        column='prop_wom', 
         cmap= cmap, 
         figsize=(20, 20), 
         linestyle='--',
         edgecolor='black', 
         legend = True,  
         scheme = "User_Defined", 
         missing_kwds= dict(color = "#DADADB",), 
         classification_kwds = dict( bins = [ 20, 30, 40, 50,  100 ] ), 
         legend_kwds=dict(  loc='upper left',
                            bbox_to_anchor=(1.01, 1),
                            fontsize='x-large',
                            title= "Women Proportion", 
                            title_fontsize = 'x-large', 
                            frameon= False )
        )

fig, ax = plt.subplots(figsize=(20, 20))
dpt_shp.plot( ax = ax )

In [22]:
dpt_shp['country'] = 'PERU'

In [23]:
ctr_shp = dpt_shp.dissolve( by = 'country')

fig, ax = plt.subplots(figsize=(20, 20))
ctr_shp.plot( ax = ax )